CREATED DIM PRODUCTS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder
    .appName("S3Test")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .getOrCreate()
)



In [2]:
silver_products = spark.read.parquet(
    "s3a://pedro-datalake-project/silver/products/"
)


In [3]:
dim_products = (
    silver_products
    .withColumn(
        "volume_cm3",
        F.col("product_length_cm")
        * F.col("product_height_cm")
        * F.col("product_width_cm")
    )
    .withColumn(
        "density",
        F.col("product_weight_g") / F.col("volume_cm3")
    )
    .withColumn(
        "is_bulky",
        F.col("volume_cm3") > 20000
    )
    .withColumn(
        "created_at",
        F.current_timestamp()
    )
)


In [4]:
dim_products.show(20, truncate=False)
dim_products.printSchema()
dim_products.count()


+--------------------------------+--------------------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------------+----------+--------------------+--------+--------------------------+
|product_id                      |product_category_name           |product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|audit_timestamp           |volume_cm3|density             |is_bulky|created_at                |
+--------------------------------+--------------------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+--------------------------+----------+--------------------+--------+--------------------------+
|1e9e8ef04dbcff4541ed26657ea517e5|perfumaria                      |40                 |287             

32951